# population analysis on ephys units across sessions

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
import sys
sys.path.insert(0, '/home/niell_Lab/Documents/github/FreelyMovingEphys')

In [3]:
from project_analysis.ephys.ephys_utils import load_ephys
from project_analysis.ephys.unit_summary import make_unit_summary
from util.paths import find

## if the data are spread across directories for individual recordings, and need to be pooled

In [ ]:
# pool data listed in csv file
df = load_ephys(r'T:\freely_moving_ephys\batch_files\written040221\exp_pool.csv')

In [ ]:
# save as one h5 file
df.to_hdf(r'T:\freely_moving_ephys\batch_files\written040221\exp_pool_all_data_writen_040421.h5', 'w')

In [ ]:
# make a pdf summary
make_unit_summary(df, r'T:\freely_moving_ephys\batch_files\written040221')

## if there is already an h5 file of pooled data

In [4]:
# load existing .h5 file
data = pd.read_hdf(r'T:\freely_moving_ephys\batch_files\written040221\exp_pool_all_data_writen_040421.h5')

In [5]:
# make a pdf summary
make_unit_summary(data, r'T:\freely_moving_ephys\batch_files\written040221')

68it [01:23,  1.11s/it]C:/Users/Niell Lab/Documents/GitHub/FreelyMovingEphys\project_analysis\ephys\unit_summary.py:191: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  unitfig_wnsrvgz.set_ylim(0,np.nanmax(tuning[:]*1.2))
178it [03:43,  2.70s/it]C:/Users/Niell Lab/Documents/GitHub/FreelyMovingEphys\project_analysis\ephys\unit_summary.py:191: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  unitfig_wnsrvgz.set_ylim(0,np.nanmax(tuning[:]*1.2))
188it [03:55,  1.20s/it]C:/Users/Niell Lab/Documents/GitHub/FreelyMovingEphys\project_analysis\ephys\unit_summary.py:191: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  unitfig_wnsrvgz.set_ylim(0,np.nanmax(tuning[:]*1.2))
192it [03:59,  1.15s/it]C:/Users/Niell Lab/Documents/GitHub/FreelyMovingEphys\project_analysis\ephys\uni

saving pdf
